In [ ]:
from time import sleep
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
import time

face_classifier = cv2.CascadeClassifier('D:\DeepLearning\Emotion\haarcascade_frontalface_default.xml')

age_model = load_model('D:\DeepLearning\Age_Gender_Ethnicity\checkPoint_model_age.h5')
gender_model = load_model('D:\DeepLearning\Age_Gender_Ethnicity\checkPoint_model_gender.h5')
ethnicity_model = load_model('D:\DeepLearning\Age_Gender_Ethnicity\checkPoint_model_ethnicity.h5')

gender_classes = ['Male', 'Famale']
ethnicity_classes = ['White', 'Black', 'Asian', 'Indian', 'Other']

prev_frame_time = 0
new_frame_time = 0

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    labels = []
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    fps = int(fps)
    fps = 'FPS: ' + str(fps)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48), interpolation = cv2.INTER_AREA)

        if np.sum([roi_gray])!=0:
            
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)
                    
            # Ethnicity
            ethnicity_pred = ethnicity_model.predict(roi)[0]
            et_label = ethnicity_classes[ethnicity_pred.argmax()]
            et_label_position = (x+h, y+h-60)
            cv2.putText(frame,et_label, et_label_position, cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            
            # FPS
            cv2.putText(frame, fps, (0, 30),  cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            
            #Gender            
            gender_pred = gender_model.predict(roi)[0]
            print()
            if gender_pred >= 0.5:
                g_lbl = gender_classes[1]
            else:
                g_lbl = gender_classes[0]
            
            g_lbl_pos = (x+h, y+h-30)
            cv2.putText(frame, g_lbl, g_lbl_pos, cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            
            # Age
            age_pred = int(age_model.predict(roi)[0])
            a_lbl_pos = (x+h, y+h)
            cv2.putText(frame, 'Age: ' + str(age_pred), a_lbl_pos, cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

        else:
            cv2.putText(frame,'No Faces',(30,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)
        
    cv2.imshow('Emotion', frame)
              
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()